## Import Libraries

In [10]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
df=pd.read_csv(r"C:\Users\myloe\OneDrive\Desktop\shared_articles.csv")

In [12]:
df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


To simplify the analysis, we are only focusing on the "CONTENT SHARED" event type and making the assumption (which may not be entirely accurate) that all articles were available throughout the entire one-year period.

In [13]:
df = df[df['eventType'] == 'CONTENT SHARED']
df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [14]:
df.describe

<bound method NDFrame.describe of        timestamp       eventType            contentId       authorPersonId  \
1     1459193988  CONTENT SHARED -4110354420726924665  4340306774493623681   
2     1459194146  CONTENT SHARED -7292285110016212249  4340306774493623681   
3     1459194474  CONTENT SHARED -6151852268067518688  3891637997717104548   
4     1459194497  CONTENT SHARED  2448026894306402386  4340306774493623681   
5     1459194522  CONTENT SHARED -2826566343807132236  4340306774493623681   
...          ...             ...                  ...                  ...   
3117  1487946604  CONTENT SHARED  9213260650272029784  3609194402293569455   
3118  1487947067  CONTENT SHARED -3295913657316686039  6960073744377754728   
3119  1488223224  CONTENT SHARED  3618271604906293310  1908339160857512799   
3120  1488300719  CONTENT SHARED  6607431762270322325 -1393866732742189886   
3121  1488307871  CONTENT SHARED  4109618890343020064  3891637997717104548   

          authorSessionId    

In [15]:
df.shape

(3047, 13)

In [16]:
df.info

<bound method DataFrame.info of        timestamp       eventType            contentId       authorPersonId  \
1     1459193988  CONTENT SHARED -4110354420726924665  4340306774493623681   
2     1459194146  CONTENT SHARED -7292285110016212249  4340306774493623681   
3     1459194474  CONTENT SHARED -6151852268067518688  3891637997717104548   
4     1459194497  CONTENT SHARED  2448026894306402386  4340306774493623681   
5     1459194522  CONTENT SHARED -2826566343807132236  4340306774493623681   
...          ...             ...                  ...                  ...   
3117  1487946604  CONTENT SHARED  9213260650272029784  3609194402293569455   
3118  1487947067  CONTENT SHARED -3295913657316686039  6960073744377754728   
3119  1488223224  CONTENT SHARED  3618271604906293310  1908339160857512799   
3120  1488300719  CONTENT SHARED  6607431762270322325 -1393866732742189886   
3121  1488307871  CONTENT SHARED  4109618890343020064  3891637997717104548   

          authorSessionId      

In [17]:
df.isnull().sum()

timestamp             0
eventType             0
contentId             0
authorPersonId        0
authorSessionId       0
authorUserAgent    2378
authorRegion       2378
authorCountry      2378
contentType           0
url                   0
title                 0
text                  0
lang                  0
dtype: int64

The "users_interactions.csv" file includes records of user interactions with shared articles. This file can be merged with the "articles_shared.csv" file based on the "contentId" column. The possible values for the "eventType" field are "VIEW", "LIKE", "COMMENT CREATED", "FOLLOW", and "BOOKMARK". These values indicate whether the user opened the article, liked it, created a comment, followed the article, or bookmarked it for future reference.

In [18]:
interactions_df = pd.read_csv(r"C:\Users\myloe\OneDrive\Desktop\users_interactions.csv")
interactions_df.head(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


we assign weights or strengths to the different types of user interactions, assuming that certain types of interactions, such as creating a comment, indicate a higher level of interest in the article than a like or a simple view. This allows us to more accurately measure user engagement and interest in the shared articles.

In [19]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])